In [2]:
%%html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>AI Customer Service Chain</title>
    <script src="https://cdn.tailwindcss.com"></script>
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700&display=swap" rel="stylesheet">
    <style>
        body { font-family: 'Inter', sans-serif; background-color: #f7f9fb; }
        .container { max-width: 800px; }
        .step-card { transition: all 0.3s ease; }
        .step-card.success { border-left: 4px solid #10b981; }
        .step-card.pending { border-left: 4px solid #f59e0b; }
        .loading-spinner { border-top-color: #3b82f6; }
    </style>
</head>
<body>

    <div class="container mx-auto p-4 md:p-8">
        <header class="text-center mb-8">
            <h1 class="text-4xl font-extrabold text-gray-900 mb-2">Customer Service AI Chain</h1>
            <p class="text-lg text-gray-500">Simulate a two-step AI triage and response flow using the Gemini API.</p>
        </header>

        <div class="bg-white p-6 md:p-8 rounded-xl shadow-2xl border border-gray-100">

            <!-- Input Area -->
            <div class="mb-6">
                <label for="customerQuery" class="block text-sm font-medium text-gray-700 mb-2">
                    Enter the Customer's Issue:
                </label>
                <textarea id="customerQuery" rows="3"
                    class="w-full p-3 border border-gray-300 rounded-lg focus:ring-blue-500 focus:border-blue-500 transition duration-150"
                    placeholder="Example: My order #4582 was supposed to be delivered yesterday but it hasn't arrived. Can you check the status?"></textarea>
            </div>

            <button id="startButton" onclick="startChain()"
                class="w-full bg-blue-600 hover:bg-blue-700 text-white font-semibold py-3 px-4 rounded-xl shadow-md transition duration-200 ease-in-out transform hover:scale-[1.01] focus:outline-none focus:ring-4 focus:ring-blue-500 focus:ring-opacity-50 disabled:bg-gray-400">
                Start Triage Chain
            </button>

            <!-- Loading Indicator -->
            <div id="loading" class="mt-6 hidden text-center">
                <div class="inline-block h-8 w-8 animate-spin rounded-full border-4 border-solid border-gray-200 border-r-transparent align-[-0.125em] motion-reduce:animate-[spin_1.5s_linear_infinite] loading-spinner"
                    role="status">
                    <span class="!absolute !-m-px !h-px !w-px !overflow-hidden !whitespace-nowrap !border-0 !p-0 ![clip:rect(0,0,0,0)]">
                        Loading...
                    </span>
                </div>
                <p class="text-gray-500 mt-2">Processing steps...</p>
            </div>

            <!-- Results Area -->
            <div id="resultsArea" class="mt-8 space-y-4">
                <h2 id="resultsTitle" class="text-2xl font-bold text-gray-800 hidden">Chain Results</h2>

                <!-- Step 1 Card -->
                <div id="step1Card" class="step-card p-4 bg-gray-50 rounded-lg shadow hidden">
                    <h3 class="font-semibold text-lg text-gray-800 flex items-center mb-2">
                        <span id="step1Icon" class="text-xl mr-2 text-gray-500">
                            <svg class="h-5 w-5 animate-spin hidden" xmlns="http://www.w3.org/2000/svg" fill="none" viewBox="0 0 24 24"><circle class="opacity-25" cx="12" cy="12" r="10" stroke="currentColor" stroke-width="4"></circle><path class="opacity-75" fill="currentColor" d="M4 12a8 8 0 018-8V0C5.373 0 0 5.373 0 12h4zm2 5.291A7.962 7.962 0 014 12H0c0 3.042 1.135 5.824 3 7.938l3-2.647z"></path></svg>
                            <span class="success-icon text-green-500 hidden">✅</span>
                        </span>
                        Step 1: Intent Classification (Structured JSON)
                    </h3>
                    <pre id="step1Output" class="text-sm bg-gray-100 p-3 rounded font-mono overflow-auto text-gray-700"></pre>
                </div>

                <!-- Step 2 Card (Final Response) -->
                <div id="step2Card" class="step-card p-4 bg-gray-50 rounded-lg shadow hidden">
                    <h3 class="font-semibold text-lg text-gray-800 flex items-center mb-2">
                        <span id="step2Icon" class="text-xl mr-2 text-gray-500">
                            <svg class="h-5 w-5 animate-spin hidden" xmlns="http://www.w3.org/2000/svg" fill="none" viewBox="0 0 24 24"><circle class="opacity-25" cx="12" cy="12" r="10" stroke="currentColor" stroke-width="4"></circle><path class="opacity-75" fill="currentColor" d="M4 12a8 8 0 018-8V0C5.373 0 0 5.373 0 12h4zm2 5.291A7.962 7.962 0 014 12H0c0 3.042 1.135 5.824 3 7.938l3-2.647z"></path></svg>
                            <span class="success-icon text-green-500 hidden">✅</span>
                        </span>
                        Step 2: Personalized Response Generation
                    </h3>
                    <p id="step2Output" class="text-gray-700 leading-relaxed"></p>
                </div>

                <div id="errorOutput" class="hidden p-4 bg-red-100 border border-red-400 text-red-700 rounded-lg" role="alert">
                    <p class="font-bold">Error:</p>
                    <p id="errorMessage"></p>
                </div>
            </div>

        </div>
    </div>

    <script type="module">
        // Standard Firebase setup boilerplate for the environment
        import { initializeApp } from "https://www.gstatic.com/firebasejs/11.6.1/firebase-app.js";
        import { getAuth, signInAnonymously, signInWithCustomToken } from "https://www.gstatic.com/firebasejs/11.6.1/firebase-auth.js";
        import { getFirestore } from "https://www.gstatic.com/firebasejs/11.6.1/firebase-firestore.js";

        // Global environment variables provided by the canvas
        const appId = typeof __app_id !== 'undefined' ? __app_id : 'default-app-id';
        const firebaseConfig = typeof __firebase_config !== 'undefined' ? JSON.parse(__firebase_config) : {};
        const initialAuthToken = typeof __initial_auth_token !== 'undefined' ? __initial_auth_token : null;

        let app, db, auth;

        // --- Gemini API Configuration ---
        const apiKey = ""; // Canvas will automatically provide the key
        const apiUrlBase = "https://generativelanguage.googleapis.com/v1beta/models/";
        const model = "gemini-2.5-flash-preview-05-20";

        // --- UI Elements ---
        const customerQueryEl = document.getElementById('customerQuery');
        const startButton = document.getElementById('startButton');
        const loadingEl = document.getElementById('loading');
        const resultsTitleEl = document.getElementById('resultsTitle');
        const errorOutputEl = document.getElementById('errorOutput');
        const errorMessageEl = document.getElementById('errorMessage');

        const step1Card = document.getElementById('step1Card');
        const step1OutputEl = document.getElementById('step1Output');
        const step1IconEl = document.getElementById('step1Icon');

        const step2Card = document.getElementById('step2Card');
        const step2OutputEl = document.getElementById('step2Output');
        const step2IconEl = document.getElementById('step2Icon');

        // --- Utility Functions ---

        /** Initializes Firebase and signs in. */
        async function initializeFirebase() {
            try {
                app = initializeApp(firebaseConfig);
                db = getFirestore(app);
                auth = getAuth(app);

                if (initialAuthToken) {
                    await signInWithCustomToken(auth, initialAuthToken);
                } else {
                    await signInAnonymously(auth);
                }
            } catch (error) {
                console.error("Firebase initialization failed:", error);
                // Non-critical for LLM functionality, but log the issue.
            }
        }

        /**
         * Generic function to make a Gemini API call with exponential backoff.
         */
        async function makeApiCall(modelName, payload, maxRetries = 5) {
            const apiUrl = `${apiUrlBase}${modelName}:generateContent?key=${apiKey}`;
            let attempt = 0;

            while (attempt < maxRetries) {
                try {
                    const response = await fetch(apiUrl, {
                        method: 'POST',
                        headers: { 'Content-Type': 'application/json' },
                        body: JSON.stringify(payload)
                    });

                    if (response.ok) {
                        return await response.json();
                    } else if (response.status === 429 && attempt < maxRetries - 1) {
                        // Rate limit error (429) -> retry
                        const delay = Math.pow(2, attempt) * 1000 + Math.random() * 1000;
                        console.warn(`Rate limit exceeded. Retrying in ${delay / 1000}s... (Attempt ${attempt + 1})`);
                        await new Promise(resolve => setTimeout(resolve, delay));
                        attempt++;
                    } else {
                        const errorBody = await response.json();
                        throw new Error(`API call failed: ${response.status} - ${errorBody.error.message || 'Unknown error'}`);
                    }
                } catch (error) {
                    if (attempt < maxRetries - 1 && error.message.includes('Failed to fetch')) {
                         // Network error -> retry
                        const delay = Math.pow(2, attempt) * 1000 + Math.random() * 1000;
                        console.warn(`Network error. Retrying in ${delay / 1000}s... (Attempt ${attempt + 1})`);
                        await new Promise(resolve => setTimeout(resolve, delay));
                        attempt++;
                    } else {
                        throw error;
                    }
                }
            }
            throw new Error('API request failed after all retries.');
        }

        /** Parses the response and extracts text or structured JSON. */
        function parseGeminiResponse(result, isJson = false) {
            const candidate = result.candidates?.[0];
            const part = candidate?.content?.parts?.[0];

            if (part?.text) {
                if (isJson) {
                    try {
                        // The model returns a string representation of the JSON, so we parse it
                        return JSON.parse(part.text);
                    } catch (e) {
                        throw new Error("Failed to parse JSON response from the model.");
                    }
                }
                return part.text;
            }

            // Handle safety filters or other issues
            const finishReason = candidate?.finishReason;
            if (finishReason && finishReason !== 'STOP') {
                throw new Error(`Generation stopped early. Reason: ${finishReason}. Ensure your prompt is safe and clear.`);
            }

            throw new Error("API response was empty or malformed.");
        }


        // --- Chaining Steps ---

        /** Step 1: Classifies the intent and extracts the key entity using a structured response. */
        async function step1_classify_intent(query) {
            const systemPrompt = "You are a Customer Support AI Classifier. Your task is to analyze the user's issue and classify its intent. You must return a single JSON object. Do not include any other text or markdown outside the JSON block.";
            const userQuery = `Classify the primary intent and extract the main entity from this customer query: "${query}"`;

            const payload = {
                contents: [{ parts: [{ text: userQuery }] }],
                systemInstruction: { parts: [{ text: systemPrompt }] },
                generationConfig: {
                    responseMimeType: "application/json",
                    responseSchema: {
                        type: "OBJECT",
                        properties: {
                            "intent": {
                                "type": "STRING",
                                "description": "The classified primary reason for the customer's query. Must be one of: ORDER_STATUS, BILLING_ISSUE, TECHNICAL_SUPPORT, PRODUCT_INQUIRY, GENERAL_FEEDBACK, OTHER."
                            },
                            "key_entity": {
                                "type": "STRING",
                                "description": "The most important number or keyword from the query, like an order number, product name, or account ID. If none is present, return 'N/A'."
                            }
                        },
                        "propertyOrdering": ["intent", "key_entity"]
                    }
                }
            };

            const result = await makeApiCall(model, payload);
            return parseGeminiResponse(result, true);
        }

        /** Step 2: Generates a final, personalized response based on the classification. */
        async function step2_generate_response(originalQuery, classification) {
            const { intent, key_entity } = classification;

            const systemPrompt = `You are Sarah, a friendly and helpful Tier 1 Customer Support Agent. Your goal is to provide a concise, single-paragraph response to the customer. Your response must address the specific intent classified as ${intent} and incorporate the key entity: ${key_entity}. If the intent is ORDER_STATUS, provide a fictional tracking update. If the intent is BILLING_ISSUE, apologize and promise immediate investigation. Use a warm, professional, and slightly casual tone.`;

            const userQuery = `Original Customer Issue: "${originalQuery}"`;

            const payload = {
                contents: [{ parts: [{ text: userQuery }] }],
                systemInstruction: { parts: [{ text: systemPrompt }] },
                // No response schema, expecting plain text
            };

            const result = await makeApiCall(model, payload);
            return parseGeminiResponse(result, false);
        }


        /** Orchestrates the two-step chain. */
        async function startChain() {
            const query = customerQueryEl.value.trim();
            if (!query) {
                alert('Please enter a customer query.');
                return;
            }

            // Reset UI
            errorOutputEl.classList.add('hidden');
            errorMessageEl.textContent = '';
            resultsTitleEl.classList.remove('hidden');
            loadingEl.classList.remove('hidden');
            startButton.disabled = true;

            step1Card.classList.remove('success');
            step1Card.classList.remove('hidden');
            step1Card.classList.add('pending');
            step1OutputEl.textContent = 'Awaiting classification...';
            step1IconEl.querySelector('.success-icon').classList.add('hidden');
            step1IconEl.querySelector('.animate-spin').classList.remove('hidden');

            step2Card.classList.add('hidden');
            step2Card.classList.remove('success');
            step2OutputEl.textContent = 'Awaiting resolution...';
            step2IconEl.querySelector('.success-icon').classList.add('hidden');

            let classificationResult = null;
            let finalResponse = null;

            try {
                // --- Step 1: Classification ---
                console.log("Starting Step 1: Intent Classification...");
                classificationResult = await step1_classify_intent(query);

                // Update UI for Step 1 success
                step1OutputEl.textContent = JSON.stringify(classificationResult, null, 2);
                step1Card.classList.remove('pending');
                step1Card.classList.add('success');
                step1IconEl.querySelector('.animate-spin').classList.add('hidden');
                step1IconEl.querySelector('.success-icon').classList.remove('hidden');
                console.log("Step 1 Complete. Classification:", classificationResult);

                // --- Step 2: Response Generation ---
                console.log("Starting Step 2: Personalized Response Generation...");
                step2Card.classList.remove('hidden');
                step2Card.classList.add('pending');
                step2IconEl.querySelector('.animate-spin').classList.remove('hidden');

                finalResponse = await step2_generate_response(query, classificationResult);

                // Update UI for Step 2 success
                step2OutputEl.textContent = finalResponse;
                step2Card.classList.remove('pending');
                step2Card.classList.add('success');
                step2IconEl.querySelector('.animate-spin').classList.add('hidden');
                step2IconEl.querySelector('.success-icon').classList.remove('hidden');
                console.log("Step 2 Complete. Final Response:", finalResponse);

            } catch (error) {
                console.error("Chain Error:", error);
                errorMessageEl.textContent = error.message;
                errorOutputEl.classList.remove('hidden');
                // Ensure all loading states are hidden on error
                step1IconEl.querySelector('.animate-spin').classList.add('hidden');
                step2IconEl.querySelector('.animate-spin').classList.add('hidden');
            } finally {
                loadingEl.classList.add('hidden');
                startButton.disabled = false;
            }
        }

        // Initialize Firebase on load
        window.onload = initializeFirebase;
    </script>
</body>
</html>